In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame(pd.read_csv('train.csv'))

In [3]:
##delete row with salary=0 or Null##
data=data[data['salary']>0]

In [4]:
##drop useless columns##
data = data.drop(['id','worker_group_name','department_name','union_name','job_group','job'],1)

In [5]:
##drop more columns##
data = data.drop(['union_code','job_group_code','worker_group_code'],1)

In [6]:
##re-order columns##
data = pd.concat([data['job_code'],data['department_code'],data['salary']],axis=1)
data=data.sort_values(by=['job_code','department_code'])

In [7]:
##translate Department_code into values & scaling##
dep_data=data.groupby(by=['department_code']).mean()

In [8]:
dep_data.index.names=[None]
dep_data['salary']/=10000

In [9]:
dep_map = dep_data.to_dict()
one_dep_map = dep_map['salary']

In [10]:
data['department_code']=data['department_code'].map(one_dep_map)

In [11]:
##translate Job_code into values & scaling##
job_data=data.groupby(by=['job_code']).mean()

In [12]:
job_data.index.names=[None]
job_data['salary']/=10000

In [13]:
job_map = job_data.to_dict()
one_job_map = job_map['salary']

In [14]:
data['job_code']=data['job_code'].map(one_job_map)

In [15]:
##count for linear regression##
from sklearn import linear_model
linreg = linear_model.LinearRegression()

In [16]:
X_train = data[['job_code','department_code']]

In [17]:
Y_train = data['salary']

In [18]:
linreg.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
linreg.intercept_

-2894.4617505259084

In [20]:
linreg.coef_

array([9856.32776132,  568.25561514])

In [21]:
##read & clear test data##
predict_data = pd.DataFrame(pd.read_csv('test.csv'))

In [22]:
predict_X = pd.concat([predict_data['job_code'],predict_data['department_code']],axis=1)

In [23]:
predict_X['job_code']=predict_X['job_code'].map(one_job_map)
predict_X['department_code']=predict_X['department_code'].map(one_dep_map)
predict_X=predict_X.fillna(0)

In [24]:
predict_result = linreg.predict(predict_X)

In [25]:
##arrange output data to desired format##
output=pd.concat([predict_data['id'],pd.DataFrame(predict_result, columns=['salary'])],axis=1)

In [26]:
output.loc[(output['salary']<0),'salary'] = 0

In [27]:
output.to_csv('output.csv', index=False)